In [1]:
import cv2
import numpy as np
from tkinter import Tk, Label
from PIL import Image, ImageTk
import threading
import winsound

def start_camera():
    global cap
    cap = cv2.VideoCapture(0)
    update_frame()

def update_frame():
    global cap, img_display, image_label, current_frame
    ret, frame = cap.read()
    if ret:
        frame = cv2.resize(frame, (380, 300))  # Resize the frame to 640x480
        current_frame = frame
        img_display = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_display = Image.fromarray(img_display)
        img_display = ImageTk.PhotoImage(img_display)
        image_label.config(image=img_display)
    image_label.after(10, update_frame)  # Update the frame every 10 milliseconds

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [vertices], 255)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines):
    if lines is None:
        return img
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1, y1), (x2, y2), (255, 0, 0), 5)
    img = cv2.addWeighted(img, 0.8, blank_image, 1.0, 0.0)
    return img

def apply_lane_detection():
    global current_frame, edge_img_display, edge_image_label
    if current_frame is not None:
        gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blur, 50, 150)

        height, width = edges.shape
        region_of_interest_vertices = [
            (0, height),
            (width / 2, height / 2),
            (width, height),
        ]
        cropped_edges = region_of_interest(edges, np.array([region_of_interest_vertices], np.int32))

        lines = cv2.HoughLinesP(
            cropped_edges,
            rho=2,
            theta=np.pi / 180,
            threshold=50,
            lines=np.array([]),
            minLineLength=40,
            maxLineGap=150
        )

        if lines is not None:
            for line in lines:
                for x1, y1, x2, y2 in line:
                    if (x1 < width / 2 and x2 < width / 2) or (x1 > width / 2 and x2 > width / 2):
                        cv2.line(current_frame, (x1, y1), (x2, y2), (0, 255, 0), 10)

        if detect_lane_deviation(lines, width):
            threading.Thread(target=alarm).start()

        edge_img_display = cv2.cvtColor(current_frame, cv2.COLOR_BGR2RGB)
        edge_img_display = Image.fromarray(edge_img_display)
        edge_img_display = ImageTk.PhotoImage(edge_img_display)
        edge_image_label.config(image=edge_img_display)

    root.after(2000, apply_lane_detection)  # Apply lane detection every 2 seconds

def detect_lane_deviation(lines, width):
    if lines is None:
        return False

    left_lane = False
    right_lane = False

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 < width / 2 and x2 < width / 2:
                left_lane = True
            if x1 > width / 2 and x2 > width / 2:
                right_lane = True

    if not left_lane or not right_lane:
        return True

    return False

def alarm():
    winsound.Beep(1000, 1000)

# Initialize the main window
root = Tk()
root.title("Road Lane Detection System")

# Image display label
image_label = Label(root)
image_label.pack()

# Edge image display label
edge_image_label = Label(root)
edge_image_label.pack()

# Start the camera and processing loop
current_frame = None
start_camera()

# Start applying lane detection every 2 seconds
apply_lane_detection()

# Run the GUI loop
root.mainloop()

# Release the camera when the application is closed
cap.release()
